In [1]:
import scipy
import scipy.sparse, scipy.stats
import numpy
import os
import re
import multiprocessing
import subprocess
import pickle
import matplotlib.pyplot
import time
import statistics
import sklearn
import statsmodels
import itertools

In [2]:
def moving_integration (values, window):
    weights = numpy.repeat(1.0, window)
    sma = numpy.convolve(values, weights, 'same')
    return sma
def moving_average (values, window):
    weights = numpy.repeat(1.0, window)/window
    sma = numpy.convolve(values, weights, 'same')
    return sma

In [3]:
with open("./testdata/peaks_tests.pi" ,"rb") as picklein:
    smoothed_diagonal,quick_peaks = pickle.load(picklein)

In [4]:
#needed to load the variables
with open("./testdata/variables.pi","rb") as picklefile:
    frag_index,frag_prop,frag_amount,valid_chroms,chroms_offsets = pickle.load(picklefile)

In [75]:
print(sum([1 if x else 0 for x in quick_peaks]))
print(frag_amount["chr1"])

60176
584662


In [76]:
import re
print(len(re.findall(r'(0+|1+)',"".join(["1" if x else "0" for x in quick_peaks])))/0.16)

63931.25


In [10]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(smoothed_diagonal,range=(0,500), bins=500)
fig.show()

In [6]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(list(itertools.compress(smoothed_diagonal, [not i for i in quick_peaks])),range=(0,500), bins=500)
ax.hist(scipy.stats.poisson.rvs(numpy.mean(smoothed_diagonal), size = 100000),range=(0,500), bins=500)
ax.hist(scipy.stats.poisson.rvs(10, size = 200000),range=(0,500), bins=500)
fig.show()

In [7]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(list(itertools.compress(smoothed_diagonal, [not i for i in quick_peaks])),range=(0,500), bins=500)
#ax.hist(scipy.stats.binom.rvs(numpy.mean(smoothed_diagonal), size = 100000),range=(0,500), bins=500)
ax.hist(scipy.stats.binom.rvs(80,0.5, size = 200000),range=(0,500), bins=500)
fig.show()

[0]